In [1]:
# carregar bibliotecas

from samgeo.text_sam import LangSAM
import glob
import os
import pandas as pd
import geopandas as gpd

In [2]:
# escolha do modelo
# vit_b(91M), vit_l(308M), vit_h(636M)

sam = LangSAM(model_type='vit_l')

final text_encoder_type: bert-base-uncased


In [3]:
def processar_imagens(caminho_pasta, formato, texto_prompt, box_t, text_t, pasta_saida):
    """
    Processa todas as imagens de uma pasta.

    Args:
      caminho_pasta: Caminho da pasta que contém as imagens.
      formato: Formato das imagens de entrada.
      texto_prompt: Prompt de texto para a predição.
      box_t: Threshold para a detecção de caixas delimitadoras.
      text_t: Threshold para a detecção de texto.
      pasta_saida: Pasta de saída dos arquivos.

    Returns:
      Nenhum.
    """

    # Obter as imagens da pasta
    imagens = glob.glob(f"{caminho_pasta}/*.{formato}")

    # Criar a pasta de saída, ignorando se ela já existir
    os.makedirs(pasta_saida, exist_ok=True)

    # Processar as imagens
    for imagem in imagens:
        # Criar o caminho da saída
        caminho_saida = f"{pasta_saida}/{os.path.basename(imagem)}_mascara.tif"
        print(f"Processando imagem: {imagem}")

        # Criar a máscara
        resultado = sam.predict(image=imagem,
                                text_prompt=texto_prompt,
                                box_threshold=box_t,
                                text_threshold=text_t,
                                output=caminho_saida)

        # Verificar se o arquivo de máscara foi criado
        if not os.path.exists(caminho_saida):
            print(f"Arquivo de máscara não encontrado: {caminho_saida}")
            continue

        print("Exportando máscara e caixas delimitadoras para shapefile.")
        # Exportar a máscara e caixas delimitadoras para shapefile
        shapefile_saida = f"{pasta_saida}/{os.path.basename(imagem)}__{texto_prompt}.shp"
        boxes_saida = f"{pasta_saida}/{os.path.basename(imagem)}__{texto_prompt}_boxes.shp"
        sam.raster_to_vector(caminho_saida, shapefile_saida)
        LangSAM.save_boxes(sam, boxes_saida)

        print("Exportação concluída.")

In [4]:
def unir_shapefiles(caminho_pasta):
    """
    Une todos os arquivos shapefile em uma pasta, exceto os chamados 'uniao.shp' e 'uniao_boxes.shp'.
    Converte todos os shapefiles para o CRS 'SIRGAS 2000 / UTM zone 22S'.

    Args:
      caminho_pasta: Caminho da pasta que contém os arquivos shapefile.

    Returns:
      Nenhum.
    """

    # Definir o CRS desejado
    crs_desejado = "EPSG:31982"  # SIRGAS 2000 / UTM zone 22S

    # Listar todos os arquivos .shp na pasta, excluindo 'uniao.shp' e 'uniao_boxes.shp'
    arquivos_shp = [f for f in glob.glob(f"{caminho_pasta}/*.shp") if not (
        os.path.basename(f) == 'uniao.shp' or os.path.basename(f) == 'uniao_boxes.shp')]

    if not arquivos_shp:
        print("Sem resultados: Nenhum arquivo .shp encontrado na pasta.")
        return

    # Separar os shapefiles em dois grupos: com e sem '_boxes'
    arquivos_boxes = [
        shp for shp in arquivos_shp if '_boxes.shp' in os.path.basename(shp)]
    arquivos_gerais = [
        shp for shp in arquivos_shp if not '_boxes.shp' in os.path.basename(shp)]

    # Função para ler, verificar CRS e unir shapefiles
    def processar_e_unir_shapefiles(arquivos, nome_saida):
        gdfs = []
        for shp in arquivos:
            gdf = gpd.read_file(shp)
            if gdf.crs is None or gdf.crs.to_string() != crs_desejado:
                gdf = gdf.to_crs(crs_desejado)
            gdfs.append(gdf)

        if gdfs:
            gdf_unido = gpd.GeoDataFrame(
                pd.concat(gdfs, ignore_index=True), crs=crs_desejado)
            caminho_saida = os.path.join(caminho_pasta, nome_saida)
            gdf_unido.to_file(caminho_saida)
            print(
                f"Shapefile '{nome_saida}' unido com sucesso e salvo em: {caminho_saida}")
        else:
            print(f"Sem arquivos para unir em '{nome_saida}'.")

    # Processar e unir os shapefiles gerais
    processar_e_unir_shapefiles(arquivos_gerais, 'uniao.shp')

    # Processar e unir os shapefiles com '_boxes'
    processar_e_unir_shapefiles(arquivos_boxes, 'uniao_boxes.shp')

In [10]:
# Uso

# caminho_pasta = 'G:\\drone_img_sam\\praia_brava_16-12-2023\\101MEDIA_georref\\'
caminho_pasta = '/Users/jac/Documents/temp/imgs_drone_praia_brava/101_georref'
formato = 'png'
texto_prompt = 'person'
box_t = 0.19
text_t = 0.19
pasta_saida = 'saida/processamento_lote/pessoas0.19'

processar_imagens(caminho_pasta, formato, texto_prompt,
                  box_t, text_t, pasta_saida)

unir_shapefiles(pasta_saida)

Processando imagem: /Users/jac/Documents/temp/imgs_drone_praia_brava/101_georref/DJI_0008_georeferenced.png
Exportando máscara e caixas delimitadoras para shapefile.
Exportação concluída.
Processando imagem: /Users/jac/Documents/temp/imgs_drone_praia_brava/101_georref/DJI_0001_georeferenced.png
Exportando máscara e caixas delimitadoras para shapefile.
Exportação concluída.
Processando imagem: /Users/jac/Documents/temp/imgs_drone_praia_brava/101_georref/DJI_0006_georeferenced.png
Exportando máscara e caixas delimitadoras para shapefile.
Exportação concluída.
Shapefile 'uniao.shp' unido com sucesso e salvo em: saida/processamento_lote/pessoas0.19/uniao.shp
Shapefile 'uniao_boxes.shp' unido com sucesso e salvo em: saida/processamento_lote/pessoas0.19/uniao_boxes.shp
